# Statistics for LWE

In [2]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ml_attack import LWEDataset, get_filename_from_params, check_secret, get_default_params, get_b_distribution

import torch

import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
import plotly.graph_objs as go

## Dataset correctness

Function to check the correctness of the algorithm (based on the error distribution):

In [38]:
# Check correctness of the LWE dataset
def check_lwe_dataset(params, num_gen=1, add_noise=True, mod_q=True, reduce=False):
    params.update({
        'num_gen': num_gen,
        'add_noise': add_noise,
        'mod_q': mod_q
    })
    dataset = LWEDataset(params)
    dataset.initialize()
    if reduce:
        dataset.reduction()

    A = dataset.get_A()
    B = dataset.get_B()
    secret = dataset.get_secret()

    e = B - np.tensordot(A, secret, axes=1)
    
    e %= params['q']
    e[e > params['q'] // 2] -= params['q']
    
    if add_noise:
        match params['error_type']:
            case 'binary':
                assert np.allclose(e, np.zeros_like(e) + 0.5, atol=0.5), "Error vector is not 0 or 1"
            case 'ternary':
                assert np.allclose(e, np.zeros_like(e), atol=1), f"Error vector is not close to zero +- 1"
            case 'cbd':
                assert np.allclose(e, np.zeros_like(e), atol=params['eta']), f"Error vector is not close to zero +- {params['eta']}"
            case _:
                raise ValueError("Unknown error type")
    else:
        assert np.allclose(e, np.zeros_like(e), atol=1e-5), "Error vector is not close to zero"

    print("All samples satisfy the LWE equation.")

# Check correctness of the LWE dataset
params = get_default_params()
params.update({
    'n': 100,
    'q': 3329,
    'k': 1,
    'secret_type': 'binary',
    'error_type': 'cbd',
    'seed': 0,
    'verbose' : True
})


check_lwe_dataset(params=params, num_gen=100, add_noise=True, mod_q=True)
check_lwe_dataset(params=params, num_gen=100, add_noise=False, mod_q=False)
check_lwe_dataset(params=params, num_gen=100, add_noise=False, mod_q=True)
check_lwe_dataset(params=params, num_gen=100, add_noise=True, mod_q=False)

#check_lwe_dataset(params=params, num_gen=2, add_noise=False, mod_q=True, reduce=True)

# This check will fail because the error propagates (but it does not become uniform)
#check_lwe_dataset(params=params, num_samples=200, add_noise=True, mod_q=True, reduce=True)


All samples satisfy the LWE equation.
All samples satisfy the LWE equation.
All samples satisfy the LWE equation.
All samples satisfy the LWE equation.


## Check B distribution and sampled B

In [ ]:
params = get_default_params()
params.update({
    'n': 100,
    'q': 3329,
    'k': 1,
    'num_gen': 1,
    'secret_type': 'cbd',
    'error_type': 'cbd',
    'seed': 0,
    
    'algos': ['BKZ2.0'],
    'bkz_block_sizes': [4, 20, 30],
    'penalty': 4,
    'verbose': True,
    
    'save_to': './../data/'
})

filename = get_filename_from_params(params)

reload = True
if os.path.exists(filename) and reload:
    print(f"Loading dataset from {filename}")
    dataset = LWEDataset.load_reduced(filename)
else:
    print(f"Generating dataset and saving to {filename}")
    dataset = LWEDataset(params)
    dataset.initialize()
    dataset.reduction()
    dataset.approximate_b()
    dataset.save_reduced()

Generating dataset and saving to ./../data/data_n_100_k_1_s_cbd_6b161.pkl


In [ ]:
values = [0, 46]
modulus = params['q']
secret = dataset.get_secret()
A = dataset.get_A()
b = dataset.get_B()

num_std = params['approximation_std']

expected_B, _, std_B = dataset.get_b_distribution()

plot_only_different = False
save_plots = False

for value in values:
    a_i = A[value]
    b_i = b[value]
    e_i = (b_i - a_i @ secret) % modulus
    e_i = e_i if e_i < modulus // 2 else e_i - modulus
    b_i = a_i @ secret + e_i

    expected_b_i = expected_B[value]
    std_b_i = std_B[value]
    sample_b_i = dataset.best_b[value]

    if plot_only_different and np.allclose(b_i, sample_b_i):
      print(f"Skipping value {value} because b_i and sample_b_i are the same.")
      continue

    x = np.linspace(expected_b_i - 4 * std_b_i, expected_b_i + 4 * std_b_i, 1000)
    pdf = norm.pdf(x, loc=expected_b_i.item(), scale=std_b_i.item())

    layout = go.Layout(
        xaxis=dict(title='R@B'),#, range=[-4e3, 4e3]),
        yaxis=dict(title='Density'),# range=[-25e-6, 400e-6]),
        bargap=0.2,
        margin=dict(l=20, r=20, t=10, b=20),
        width=550,
        height=400,
        showlegend=False,
        title=None
    )

    fig = go.Figure(layout=layout)

    # Normal Distribution
    fig.add_trace(go.Scatter(x=x, y=pdf, mode='lines', name='Normal Distribution', line=dict(color='blue')))

    # ±4 std
    fig.add_vline(x=expected_b_i.item() + num_std * std_b_i.item(), line=dict(color='yellow', dash='dash'), name='+4 std')
    fig.add_vline(x=expected_b_i.item() - num_std * std_b_i.item(), line=dict(color='yellow', dash='dash'), name='-4 std')

    # Gray background lines
    mod_b_i = b_i % modulus
    C_min = int(x.min() / modulus) - 1
    C_max = int(x.max() / modulus)
    C_values = np.arange(C_min, C_max + 1)
    for C in C_values:
      line_value = mod_b_i + C * modulus
      fig.add_vline(x=line_value, line=dict(color='gray', dash='dash'), opacity=0.5)

    # Candidate b values
    candidates = dataset.b_candidates[value]
    probs = dataset.b_probs[value]
    print(f"Candidates: {candidates},\nProbs: {probs}")

    for i, (candidate, prob) in enumerate(zip(candidates, probs)):
      fig.add_vline(x=candidate, line=dict(color='green', dash='dash'))

    # Observed b_0
    fig.add_vline(x=b_i.item(), line=dict(color='red', dash='dash'), name='Observed b_0')

    # Best candidate
    fig.add_vline(x=sample_b_i, line=dict(color='orange', dash='dash'), name='Best candidate')

    fig.show()

    if save_plots:
        fig.write_image(f"B_approx_{value}.pdf")


Candidates: [-20744. -17415. -14086. -10757.  -7428.  -4099.   -770.   2559.   5888.
   9217.  12546.  15875.  19204.  22533.],
Probs: [0.0151426  0.02900192 0.04959182 0.0757095  0.10319241 0.12557469
 0.13643115 0.13233722 0.11460603 0.08861148 0.06116874 0.03769868
 0.02074339 0.01019038]


Candidates: [-22120. -18791. -15462. -12133.  -8804.  -5475.  -2146.   1183.   4512.
   7841.  11170.  14499.  17828.  21157.],
Probs: [0.01119473 0.02246952 0.0402653  0.06442067 0.0920188  0.11735049
 0.13361348 0.1358228  0.1232685  0.09988231 0.07225736 0.04666944
 0.02691164 0.01385494]


## Reduction

In [46]:
A = dataset.get_A()
B = dataset.get_B()
secret = dataset.get_secret()

# Check if A @ secret == B
A_s = A @ secret

layout = go.Layout(
        yaxis=dict(title='Frequency'),
 #       xaxis=dict(range=[-3e3, 3e3]),
        bargap=0.2,
        margin=dict(l=20, r=20, t=10, b=20),
        width=550,
        height=400,
        showlegend=False,
        title=None
    )

# Plot distribution of A_s
fig1 = go.Figure(layout=layout)
fig1.add_trace(go.Histogram(
  x=A_s.flatten(),
  xbins=dict(start=int(A_s.min()), end=int(A_s.max()) + 1, size=1000),
  marker=dict(color='rgba(100, 150, 255, 0.7)', line=dict(color='black', width=1))
))
fig1.update_layout(
  xaxis_title='A @ secret',
)
fig1.show()

# Compute error vector e
e = B - A_s
e %= params['q']
e[e > params['q'] // 2] -= params['q']

# Plot distribution of Error Vector (e)
fig2 = go.Figure(layout=layout)
fig2.add_trace(go.Histogram(
  x=e.flatten(),
  xbins=dict(start=int(e.min()), end=int(e.max()) + 1, size=100),
  marker=dict(color='rgba(255, 100, 100, 0.7)', line=dict(color='black', width=1))
))
fig2.update_layout(
  xaxis_title='Error',
  xaxis=dict(range=[-800, 900])
)
fig2.show()

# Plot distribution of B
fig3 = go.Figure(layout=layout)
fig3.add_trace(go.Histogram(
  x=B.flatten(),
  xbins=dict(start=int(B.min()), end=int(B.max()) + 1, size=200),
  marker=dict(color='rgba(100, 255, 100, 0.7)', line=dict(color='black', width=1))
))
fig3.update_layout(
  xaxis_title='B mod q',
)
fig3.show()

save_plots = True
if save_plots:
    fig1.write_image("A_s_cbd_distr.pdf")
    fig2.write_image("error_cbd_distr.pdf")
    fig3.write_image("B_cbd_distr.pdf")

In [42]:
dataset.approximate_b()

in_candidates = 0
exact_candidates = 0

secret = dataset.get_secret()
A = dataset.get_A()
B = dataset.get_B()

for i in range(params['n']):
    a_i = A[i]
    b_i = B[i]
    e_i = (b_i - a_i @ secret) % dataset.mlwe.q
    e_i = e_i if e_i <= dataset.mlwe.q // 2 else e_i - dataset.mlwe.q
    true_b = a_i @ secret + e_i

    candidates = dataset.b_candidates[i]
    probs = dataset.b_probs[i]

    if true_b in candidates:
        in_candidates += 1

    if true_b == candidates[np.argmax(probs)]:
        exact_candidates += 1

length = params['n']
print(f"True B in candidate set: {in_candidates} / {length} ({100 * in_candidates / length:.2f}%)")
print(f"True B is the best candidate: {exact_candidates} / {length} ({100 * exact_candidates / length:.2f}%)")

True B in candidate set: 97 / 100 (97.00%)
True B is the best candidate: 9 / 100 (9.00%)


## A distribution

In [16]:
import numpy as np

# Parameters setup
num_trials = 10000
stds = []
params = {
    'n': 100,       # matrix dimension
    'q': 3329,      # modulus
}

# Fixed random matrix R (n x n)
random_matrix = np.random.randint(0, params['q'], size=(params['n'], params['n']))

# Precompute row norms for theoretical calculation
row_norms = np.linalg.norm(random_matrix, axis=1)  # ||r_i|| for each row
average_squared_row_norm = np.mean(row_norms**2)   # E[||r_i||^2]

# Theoretical std calculation (using our derivation)
theoretical_std = (params['q'] / np.sqrt(12)) * np.sqrt(average_squared_row_norm)

for _ in range(num_trials):
    # Generate random A matrix (n x n) with uniform entries in [0, q-1]
    A_tmp = np.random.randint(0, params['q'], size=(params['n'], params['n']))
    
    # Compute matrix product R @ A
    product = random_matrix @ A_tmp
    
    # Compute std of all entries in the product matrix
    stds.append(np.std(product))

# Empirical mean of stds
mean_empirical_std = np.mean(stds)

print(f"Empirical mean std of R@A: {mean_empirical_std:.2f}")
print(f"Theoretical std: {theoretical_std:.2f}")
print(f"Ratio (empirical/theoretical): {mean_empirical_std/theoretical_std:.4f}")

Empirical mean std of R@A: 23311241.08
Theoretical std: 18565074.63
Ratio (empirical/theoretical): 1.2557


In [ ]:
from ml_attack.utils import get_vector_distribution

params = {
  'n': 100,
  'eta': 3
}

delta_0 = 1.01
q = 3329
w = 4
R_norm = 100

p = 0.8


float(get_vector_distribution(params, "cbd")[2] / get_vector_distribution(params, "cbd")[2])

1.0